In [2]:
import joblib
final = joblib.load('model.pkl')
import pandas as pd
import numpy as np


In [6]:

def map_weight_class(row):
    if "Women's" in row['weight_class'] or "Strawweight" in row['weight_class']:
        return None
        
    elif "Flyweight" in row['weight_class']:
        return 0
    elif "Bantamweight" in row['weight_class']:
        return 1
    elif "Featherweight" in row['weight_class']:
        return 2
    elif "Flyweight" in row['weight_class']:
        return 3
    elif "Lightweight" in row['weight_class']:
        return 4
    elif "Welterweight" in row['weight_class']:
        return 5
    elif "Middleweight" in row['weight_class']:
        return 6
    elif "Light Heavyweight" in row['weight_class']:
        return 7
        
    elif "Heavyweight" in row['weight_class']:
        return 8
    elif "Catch Weight" in row['weight_class']:
        return 9
    elif "Open Weight" in row['weight_class']:
        return None
    else:
        print(row['weight_class'], row['fights_url'])
        return None
        
def Predictss(fighter1_name,fighter2_name,weight_class):
    import pandas as pd
    import numpy as np
    import sklearn
    fighter_data = pd.read_csv('ufc_fighters_median.csv')
    fighter_data = fighter_data.drop(columns=['fighter_weight_lbs', 'fighter_reach_cm', 'fighter_stance',"fighter_dob"])
    nf = pd.read_csv('ufc_fighters_median.csv')
    fighter_data["fighter_dob"] = nf["fighter_dob"]
    fighter_data["median_ctrl_time"] = fighter_data["median_ctrl_time"].bfill()
    fighter_data["fighter_str_acc_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_str_acc_%"])))
    fighter_data["fighter_str_def_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_str_def_%"])))
    fighter_data["fighter_td_acc_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_td_acc_%"] - 50)))
    fighter_data["fighter_td_def_%"] =1 / (1 + np.exp(-0.1 * (fighter_data["fighter_td_def_%"] - 50)))
    


    from datetime import datetime


    global final
    
    
    new = pd.DataFrame()
    today = datetime.now()
    if 'fighter_dob' in fighter_data.columns:
        fighter_data['fighter_dob'] = fighter_data['fighter_dob'].apply(
            lambda dob: today.year - pd.to_datetime(dob).year - (
                (today.month, today.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day)
            )
        )
    
    
    fighter_data = fighter_data.rename(columns={'fighter_dob': 'fighter_age_during'})

    # Prepare data for the new instance
    f1 = fighter_data[fighter_data["fighter_name"] == fighter1_name]
    f2 = fighter_data[fighter_data["fighter_name"] == fighter2_name]
    
   
    to_a = ["age_during"]
    to_a2 = ["fighter_wins", "fighter_losses", "fighter_slpm", "fighter_sapm",
             "median_knockdowns", "median_takedowns", "median_clinchs", 
             "median_ctrl_time", "median_total_fight_time", "median_submissions", 
             "median_reversals", "median_head_strikes", "median_body_strikes", 
             "median_leg_strikes", "median_grounds"]
    
    row_data = {}
    for col in to_a:
        row_data[f'f1_{col}'] = f1[f'fighter_{col}'].item()
        row_data[f'f2_{col}'] = f2[f'fighter_{col}'].item()
    for col in to_a2:
        row_data[f'f1_{col}'] = f1[col].item()
        row_data[f'f2_{col}'] = f2[col].item()
    
    new = pd.DataFrame(row_data, index=[0])
    new['weight_class'] = weight_class
    new['weight_class'] = new.apply(map_weight_class, axis=1)
    
    # Align with training data columns
    new = new.reindex(columns=['weight_class', 'f1_age_during', 'f2_age_during',
       'f1_median_knockdowns', 'f2_median_knockdowns', 'f1_median_takedowns',
       'f2_median_takedowns', 'f1_median_clinchs', 'f2_median_clinchs',
       'f1_median_ctrl_time', 'f2_median_ctrl_time',
       'f1_median_total_fight_time', 'f2_median_total_fight_time',
       'f1_median_submissions', 'f2_median_submissions', 'f1_median_reversals',
       'f2_median_reversals', 'f1_median_head_strikes',
       'f2_median_head_strikes', 'f1_median_body_strikes',
       'f2_median_body_strikes', 'f1_median_leg_strikes',
       'f2_median_leg_strikes', 'f1_median_grounds', 'f2_median_grounds',
       'f1_fighter_wins', 'f1_fighter_losses', 'f1_fighter_slpm',
       'f1_fighter_sapm', 'f2_fighter_wins', 'f2_fighter_losses',
       'f2_fighter_slpm', 'f2_fighter_sapm'], fill_value=0)
    
    # Convert to NumPy array for prediction
    new_array = np.array(new) # Ensure numerical data only
    
    # Predict probabilities and outcome
    prediction_probs = final.predict_proba(new)
    single_pred = final.predict(new)
    
    probs = list(prediction_probs[0])  # Unpack probabilities
    winners = {fighter1_name: probs[0], fighter2_name: probs[1]}
    

    new = pd.DataFrame()
    today = datetime.now()
    if 'fighter_dob' in fighter_data.columns:
        fighter_data['fighter_dob'] = fighter_data['fighter_dob'].apply(
            lambda dob: today.year - pd.to_datetime(dob).year - (
                (today.month, today.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day)
            )
        )
    
    
    fighter_data = fighter_data.rename(columns={'fighter_dob': 'fighter_age_during'})
    
    # Prepare data for the new instance
    f1 = fighter_data[fighter_data["fighter_name"] == fighter2_name]
    f2 = fighter_data[fighter_data["fighter_name"] == fighter1_name]
   
    to_a = ["age_during"]
    to_a2 = ["fighter_wins", "fighter_losses", "fighter_slpm", "fighter_sapm",
             "median_knockdowns", "median_takedowns", "median_clinchs", 
             "median_ctrl_time", "median_total_fight_time", "median_submissions", 
             "median_reversals", "median_head_strikes", "median_body_strikes", 
             "median_leg_strikes", "median_grounds"]
    
    row_data = {}
    for col in to_a:
        row_data[f'f1_{col}'] = f1[f'fighter_{col}'].item()
        row_data[f'f2_{col}'] = f2[f'fighter_{col}'].item()
    for col in to_a2:
        row_data[f'f1_{col}'] = f1[col].item()
        row_data[f'f2_{col}'] = f2[col].item()
    
    new = pd.DataFrame(row_data, index=[0])
    new['weight_class'] = weight_class
    new['weight_class'] = new.apply(map_weight_class, axis=1)
    
    # Align with training data columns
    new = new.reindex(columns=['weight_class', 'f1_age_during', 'f2_age_during',
       'f1_median_knockdowns', 'f2_median_knockdowns', 'f1_median_takedowns',
       'f2_median_takedowns', 'f1_median_clinchs', 'f2_median_clinchs',
       'f1_median_ctrl_time', 'f2_median_ctrl_time',
       'f1_median_total_fight_time', 'f2_median_total_fight_time',
       'f1_median_submissions', 'f2_median_submissions', 'f1_median_reversals',
       'f2_median_reversals', 'f1_median_head_strikes',
       'f2_median_head_strikes', 'f1_median_body_strikes',
       'f2_median_body_strikes', 'f1_median_leg_strikes',
       'f2_median_leg_strikes', 'f1_median_grounds', 'f2_median_grounds',
       'f1_fighter_wins', 'f1_fighter_losses', 'f1_fighter_slpm',
       'f1_fighter_sapm', 'f2_fighter_wins', 'f2_fighter_losses',
       'f2_fighter_slpm', 'f2_fighter_sapm'], fill_value=0)
    new_array = np.array(new)
    # Convert to NumPy array for prediction
   
    prediction_probs = final.predict_proba(new)  
    single_pred =final.predict(new)  
    probs = list(prediction_probs)
    winners[fighter1_name] += probs[0][1]
    winners[fighter2_name] += probs[0][0]
    winners[fighter1_name]= winners[fighter1_name]/2
    winners[fighter2_name] =winners[fighter2_name]/2
    print((max(winners, key=winners.get)))
    print (winners)


In [ ]:
Predictss("Fares Ziam","Mike Davis","Lightweight")



ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
to_pred = pd.read_csv("ufc-master (1).csv",nrows = 15)

In [ ]:

def map_weight_class(row):
    if "Women's" in row['weight_class'] or "Strawweight" in row['weight_class']:
        return None
    elif "Flyweight" in row['weight_class']:
        return 0
    elif "Bantamweight" in row['weight_class']:
        return 1
    elif "Featherweight" in row['weight_class']:
        return 2
    elif "Flyweight" in row['weight_class']:
        return 3
    elif "Lightweight" in row['weight_class']:
        return 4
    elif "Welterweight" in row['weight_class']:
        return 5
    elif "Middleweight" in row['weight_class']:
        return 6
    elif "Light Heavyweight" in row['weight_class']:
        return 7
        
    elif "Heavyweight" in row['weight_class']:
        return 8
    elif "Catch Weight" in row['weight_class']:
        return 9
    elif "Open Weight" in row['weight_class']:
        return None
    else:
        print(row['weight_class'], row['fights_url'])
        return None
        
def Predictss_csv(file):
    import pandas as pd
    import numpy as np
    import sklearn
    fighter_data = pd.read_csv('ufc_fighters_median.csv')
    fighter_data = fighter_data.drop(columns=['fighter_weight_lbs', 'fighter_reach_cm', 'fighter_stance',"fighter_dob"])
    nf = pd.read_csv('ufc_fighters_median.csv')
    fighter_data["fighter_dob"] = nf["fighter_dob"]
    fighter_data["median_ctrl_time"] = fighter_data["median_ctrl_time"].bfill()
    fighter_data["fighter_str_acc_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_str_acc_%"])))
    fighter_data["fighter_str_def_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_str_def_%"])))
    fighter_data["fighter_td_acc_%"] = 1 / (1 + np.exp(-0.1 * (fighter_data["fighter_td_acc_%"] - 50)))
    fighter_data["fighter_td_def_%"] =1 / (1 + np.exp(-0.1 * (fighter_data["fighter_td_def_%"] - 50)))
    


    from datetime import datetime


    global final
    
    
    new = pd.DataFrame()
    today = datetime.now()
    if 'fighter_dob' in fighter_data.columns:
        fighter_data['fighter_dob'] = fighter_data['fighter_dob'].apply(
            lambda dob: today.year - pd.to_datetime(dob).year - (
                (today.month, today.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day)
            )
        )
    
    
    fighter_data = fighter_data.rename(columns={'fighter_dob': 'fighter_age_during'})
    for row in file.iterrows():
        try:
            # Prepare data for the new instance
            f1 = fighter_data[fighter_data["fighter_name"] == row[1]["RedFighter"]]
            f2 = fighter_data[fighter_data["fighter_name"] == row[1]["BlueFighter"]]
            
           
            to_a = ["age_during"]
            to_a2 = ["fighter_wins", "fighter_losses", "fighter_slpm", "fighter_sapm",
                     "median_knockdowns", "median_takedowns", "median_clinchs", 
                     "median_ctrl_time", "median_total_fight_time", "median_submissions", 
                     "median_reversals", "median_head_strikes", "median_body_strikes", 
                     "median_leg_strikes", "median_grounds"]
            
            row_data = {}
    
            for col in to_a:
                row_data[f'f1_{col}'] = f1[f'fighter_{col}'].item()
                row_data[f'f2_{col}'] = f2[f'fighter_{col}'].item()
            for col in to_a2:
                row_data[f'f1_{col}'] = f1[col].item()
                row_data[f'f2_{col}'] = f2[col].item()
            new = pd.DataFrame(row_data, index=[0])
            new['weight_class'] = "Lightweight"
            new['weight_class'] = new.apply(map_weight_class, axis=1)
            
            # Align with training data columns
            new = new.reindex(columns=['weight_class', 'f1_age_during', 'f2_age_during',
               'f1_median_knockdowns', 'f2_median_knockdowns', 'f1_median_takedowns',
               'f2_median_takedowns', 'f1_median_clinchs', 'f2_median_clinchs',
               'f1_median_ctrl_time', 'f2_median_ctrl_time',
               'f1_median_total_fight_time', 'f2_median_total_fight_time',
               'f1_median_submissions', 'f2_median_submissions', 'f1_median_reversals',
               'f2_median_reversals', 'f1_median_head_strikes',
               'f2_median_head_strikes', 'f1_median_body_strikes',
               'f2_median_body_strikes', 'f1_median_leg_strikes',
               'f2_median_leg_strikes', 'f1_median_grounds', 'f2_median_grounds',
               'f1_fighter_wins', 'f1_fighter_losses', 'f1_fighter_slpm',
               'f1_fighter_sapm', 'f2_fighter_wins', 'f2_fighter_losses',
               'f2_fighter_slpm', 'f2_fighter_sapm'], fill_value=0)
            
            # Convert to NumPy array for prediction
            new_array = np.array(new) # Ensure numerical data only
            
            # Predict probabilities and outcome
            prediction_probs = final.predict_proba(new)
            single_pred = final.predict(new)
            
            probs = list(prediction_probs[0])  # Unpack probabilities
            winners = {row[1]["RedFighter"]: probs[0], row[1]["BlueFighter"]: probs[1]}
            
         
            new = pd.DataFrame()
            today = datetime.now()
            if 'fighter_dob' in fighter_data.columns:
                fighter_data['fighter_dob'] = fighter_data['fighter_dob'].apply(
                    lambda dob: today.year - pd.to_datetime(dob).year - (
                        (today.month, today.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day)
                    )
                )
            
            
            fighter_data = fighter_data.rename(columns={'fighter_dob': 'fighter_age_during'})
            
            # Prepare data for the new instance
            f1 = fighter_data[fighter_data["fighter_name"] == row[1]["BlueFighter"]]
            f2 = fighter_data[fighter_data["fighter_name"] == row[1]["RedFighter"]]
           
            to_a = ["age_during"]
            to_a2 = ["fighter_wins", "fighter_losses", "fighter_slpm", "fighter_sapm",
                     "median_knockdowns", "median_takedowns", "median_clinchs", 
                     "median_ctrl_time", "median_total_fight_time", "median_submissions", 
                     "median_reversals", "median_head_strikes", "median_body_strikes", 
                     "median_leg_strikes", "median_grounds"]
    
            row_data = {}
            for col in to_a:
                row_data[f'f1_{col}'] = f1[f'fighter_{col}'].item()
                row_data[f'f2_{col}'] = f2[f'fighter_{col}'].item()
            for col in to_a2:
                row_data[f'f1_{col}'] = f1[col].item()
                row_data[f'f2_{col}'] = f2[col].item()
      
            new = pd.DataFrame(row_data, index=[0])
            new['weight_class'] = "Lightweight"

            new['weight_class'] = new.apply(map_weight_class, axis=1)
            
            # Align with training data columns
            new = new.reindex(columns=['weight_class', 'f1_age_during', 'f2_age_during',
               'f1_median_knockdowns', 'f2_median_knockdowns', 'f1_median_takedowns',
               'f2_median_takedowns', 'f1_median_clinchs', 'f2_median_clinchs',
               'f1_median_ctrl_time', 'f2_median_ctrl_time',
               'f1_median_total_fight_time', 'f2_median_total_fight_time',
               'f1_median_submissions', 'f2_median_submissions', 'f1_median_reversals',
               'f2_median_reversals', 'f1_median_head_strikes',
               'f2_median_head_strikes', 'f1_median_body_strikes',
               'f2_median_body_strikes', 'f1_median_leg_strikes',
               'f2_median_leg_strikes', 'f1_median_grounds', 'f2_median_grounds',
               'f1_fighter_wins', 'f1_fighter_losses', 'f1_fighter_slpm',
               'f1_fighter_sapm', 'f2_fighter_wins', 'f2_fighter_losses',
               'f2_fighter_slpm', 'f2_fighter_sapm'], fill_value=0)
            new_array = np.array(new)
            # Convert to NumPy array for prediction
            prediction_probs = final.predict_proba(new)  
            single_pred =final.predict(new)  
            probs = list(prediction_probs)
            winners[row[1]["RedFighter"]] += probs[0][1]
            winners[row[1]["BlueFighter"]] += probs[0][0]
            winners[row[1]["RedFighter"]]= winners[row[1]["RedFighter"]]/2
            winners[row[1]["BlueFighter"]] =winners[row[1]["BlueFighter"]]/2
            print((max(winners, key=winners.get)))
            print (winners)
        except:
            pass
        


In [ ]:
print(Predictss_csv(to_pred))

Deiveson Figueiredo
{'Petr Yan': 0.4609348635463162, 'Deiveson Figueiredo': 0.5390651364536838}
Yan Xiaonan
{'Yan Xiaonan': 0.6245307954137735, 'Tabatha Ricci': 0.3754692045862264}
Muslim Salikhov
{'Song Kenan': 0.42232973020443326, 'Muslim Salikhov': 0.5776702697955667}
Wang Cong
{'Wang Cong': 0.7225292525390441, 'Gabriella Fernandes': 0.2774707474609559}
Carlos Ulberg
{'Volkan Oezdemir': 0.39164937536165323, 'Carlos Ulberg': 0.6083506246383468}
Zhang Mingyang
{'Zhang Mingyang': 0.882083569748565, 'Ozzy Diaz': 0.11791643025143492}
Baergeng Jieleyisi
{'Baergeng Jieleyisi': 0.7892759246142181, 'SuYoung You': 0.21072407538578186}
Feng Xiaocan
{'Shi Ming': 0.43312636019970707, 'Feng Xiaocan': 0.5668736398002929}
Carlos Hernandez
{'Nyamjargal Tumendemberel': 0.49397969480036885, 'Carlos Hernandez': 0.5060203051996311}
Lone'er Kavanagh
{"Lone'er Kavanagh": 0.5, 'Jose Ochoa': 0.5}
Xiao Long
{'Xiao Long': 0.6259584943955527, 'Quang Le': 0.37404150560444727}
Nikolas Motta
{'Maheshate': 0.30539

In [ ]:
for row in to_pred.iterrows():
    print(row[1]["RedFighter"])

Petr Yan
Yan Xiaonan
Song Kenan
Wang Cong
Volkan Oezdemir
Zhang Mingyang
Baergeng Jieleyisi
Kiru Sahota
Shi Ming
Nyamjargal Tumendemberel
Lone'er Kavanagh
Xiao Long
Maheshate
Jon Jones
Charles Oliveira
